In [18]:
import os as os

import tensorflow as tf

from sklearn.model_selection import train_test_split

## Data processing

In [66]:
def convertLabel(label): #gives a nb to the aminoacid
    aminoacidList = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE",
                    "LEU", "LYS", "MET", "PHE", "PRO", "PYL", "SEL", "SER", "THR", "TRP", "TYR", "VAL"]
    for i in range (0, 22):
        if label == aminoacidList[i]:
            return i
    print("ERROR")

In [67]:
def load_data(filename):


    labelList = []

    with open(filename) as f:

        nbOfSamples = int(f.readline())
        i = 0

        shapes = f.readline() # length of sequence + nb of features
        input_shape = int(shapes.split(",")[1])

        # inputs
        distancesList = np.zeros((nbOfSamples, input_shape))
        previousResiduesList = np.zeros((nbOfSamples, 7))
        
        # outputs
        labelList = np.zeros(nbOfSamples)
        


        while i < nbOfSamples:
            # Extracting labels
            sequence = f.readline().split(",")
            labelList[i] = convertLabel(sequence[8])
            previousResiduesList[i, :] = np.array([convertLabel(sequence[i]) for i in range(0, 7)])

            # Extracting distances
            distancesString = f.readline()
            distancesArray = np.array([float(s) for s in distancesString.split(",")])
            distancesList[i, :] = distancesArray

            # empty line
            f.readline()

            i += 1
    return input_shape, distancesList, previousResiduesList, labelList

## Creating models

### Single input model

In [68]:
def single_input_model():
    # activation function: leaky ReLU
    leakyrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01, max_value=None, threshold=0)

    # creation du reseau de neurones
    model = tf.keras.models.Sequential([

        # hidden layer
        tf.keras.layers.Dense(units = 105, activation = leakyrelu, input_shape = (105,)),
      #  keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
       # keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        #keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        tf.keras.layers.Dense(units = 100, activation = leakyrelu),
        #keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 75, activation = leakyrelu),
        #keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 53, activation = leakyrelu),
        #tf.keras.layers.Dropout(rate=0.2),

        # final layer
        tf.keras.layers.Dense(units = 22, activation = tf.nn.softmax),
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

In [69]:
# Define checkppoints
single_input_checkpoint_path = "Single input model/checkpoints/checkpoint.ckpt"
single_input_checkpoint_dir = os.path.dirname(single_input_checkpoint_path)

single_input_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    single_input_checkpoint_path, save_weights_only=True, verbose=1,
    # save weights, every 10-epoch
    period=10)

single_input_latest = tf.train.latest_checkpoint(single_input_checkpoint_dir)

### Double input model

In [82]:
def double_input_model():
    # activation function: leaky ReLU
    leakyrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01, max_value=None, threshold=0)

    # distances branch
    distancesBranch = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units = 105, activation = leakyrelu, input_shape = (105,)),
        #tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        #tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        #tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 100, activation = leakyrelu),
       # tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 75, activation = leakyrelu),
        #tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 53, activation = leakyrelu),
        #tf.keras.layers.Dropout(rate=0.2)
    ])
    
    # residues branch   
    residuesBranch = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units = 7, activation = leakyrelu, input_shape = (7,))
    ])
    
    # concatenation
    combinedInput = tf.keras.layers.concatenate([distancesBranch.output, residuesBranch.output])

    x = tf.keras.layers.Dense(units = 30, activation = leakyrelu, input_shape = (59,))(combinedInput)
    x = tf.keras.layers.Dense(units = 22, activation = tf.nn.softmax)(x)
    
    model = tf.keras.models.Model(inputs = [distancesBranch.input, residuesBranch.input], outputs = x)

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

In [71]:
# Define checkppoints
double_input_checkpoint_path = "Double input model/checkpoints/checkpoint.ckpt"
double_input_checkpoint_dir = os.path.dirname(double_input_checkpoint_path)

double_input_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    double_input_checkpoint_path, save_weights_only=True, verbose=1,
    # save weights, every 10-epoch
    period=10)

double_input_latest = tf.train.latest_checkpoint(double_input_checkpoint_dir)

## Saving models

In [72]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

## Training

### Importing data

In [73]:
# Data processing
input_shape, distances, previousResidues, output = load_data("Data/matrices.csv")


distances_train, distances_test, residues_train, residues_test, y_train, y_test = train_test_split(
    distances, previousResidues, output, test_size=0.2, shuffle = False, random_state=2)

### Training the single input model

#### Creation

In [60]:
single_input_sess = tf.Session()
keras.backend.set_session(sess)

single_input_model = single_input_model()

if single_input_latest: single_input_model.load_weights(single_input_latest)

#### Training

In [61]:
single_input_model.fit(distances_train, y_train,
                       validation_data = (distances_test, y_test),
                       callbacks = [single_input_cp_callback], epochs = 20)

Train on 976 samples, validate on 245 samples
Epoch 1/20
976/976 [==============================] - 1s 665us/sample - loss: 3.0855 - acc: 0.0840 - val_loss: 2.9130 - val_acc: 0.0980
Epoch 2/20
976/976 [==============================] - 0s 194us/sample - loss: 2.9139 - acc: 0.0984 - val_loss: 2.8956 - val_acc: 0.1143
Epoch 3/20
976/976 [==============================] - 0s 199us/sample - loss: 2.8903 - acc: 0.1117 - val_loss: 2.8457 - val_acc: 0.0857
Epoch 4/20
976/976 [==============================] - 0s 192us/sample - loss: 2.8318 - acc: 0.1168 - val_loss: 2.8222 - val_acc: 0.1224
Epoch 5/20
976/976 [==============================] - 0s 194us/sample - loss: 2.8006 - acc: 0.1168 - val_loss: 2.7707 - val_acc: 0.1143
Epoch 6/20
976/976 [==============================] - 0s 196us/sample - loss: 2.7872 - acc: 0.1322 - val_loss: 2.8044 - val_acc: 0.0939
Epoch 7/20
976/976 [==============================] - 0s 200us/sample - loss: 2.7843 - acc: 0.1260 - val_loss: 2.7683 - val_acc: 0.1347
Ep


Consider using a TensorFlow optimizer from `tf.train`.


976/976 [==============================] - 1s 1ms/sample - loss: 2.7168 - acc: 0.1547 - val_loss: 2.7337 - val_acc: 0.1265
Epoch 11/20
976/976 [==============================] - 0s 186us/sample - loss: 2.6971 - acc: 0.1455 - val_loss: 2.6666 - val_acc: 0.1429
Epoch 12/20
976/976 [==============================] - 0s 203us/sample - loss: 2.6285 - acc: 0.1732 - val_loss: 2.7084 - val_acc: 0.1510
Epoch 13/20
976/976 [==============================] - 0s 203us/sample - loss: 2.5860 - acc: 0.1752 - val_loss: 2.6287 - val_acc: 0.1673
Epoch 14/20
976/976 [==============================] - 0s 207us/sample - loss: 2.6110 - acc: 0.1814 - val_loss: 2.6508 - val_acc: 0.1551
Epoch 15/20
976/976 [==============================] - 0s 210us/sample - loss: 2.5639 - acc: 0.1855 - val_loss: 2.5885 - val_acc: 0.1878
Epoch 16/20
976/976 [==============================] - 0s 202us/sample - loss: 2.5211 - acc: 0.2080 - val_loss: 2.6334 - val_acc: 0.1510
Epoch 17/20
976/976 [==============================] - 


Consider using a TensorFlow optimizer from `tf.train`.


976/976 [==============================] - 1s 834us/sample - loss: 2.3321 - acc: 0.2449 - val_loss: 2.5877 - val_acc: 0.1633


#### Saving

In [85]:
single_input_model.save("Single input model/single_input_model.h5")

AttributeError: 'function' object has no attribute 'save'

### Training the double input model

#### Creation

In [83]:


double_input_model = double_input_model()

if double_input_latest: double_input_model.load_weights(double_input_latest)

#### Training

In [84]:
double_input_model.fit([distances_train, residues_train], y_train,
                       validation_data = ([distances_test, residues_test], y_test),
                       callbacks = [double_input_cp_callback], epochs = 20)

Train on 976 samples, validate on 245 samples
Epoch 1/20
976/976 [==============================] - 1s 1ms/sample - loss: 4.4298 - acc: 0.0441 - val_loss: 3.2175 - val_acc: 0.0776
Epoch 2/20
976/976 [==============================] - 0s 181us/sample - loss: 3.1670 - acc: 0.0758 - val_loss: 3.0022 - val_acc: 0.1102
Epoch 3/20
976/976 [==============================] - 0s 180us/sample - loss: 2.9788 - acc: 0.1066 - val_loss: 2.9148 - val_acc: 0.0857
Epoch 4/20
976/976 [==============================] - 0s 181us/sample - loss: 2.9207 - acc: 0.1055 - val_loss: 2.9224 - val_acc: 0.1265
Epoch 5/20
976/976 [==============================] - 0s 236us/sample - loss: 2.8767 - acc: 0.1260 - val_loss: 2.8458 - val_acc: 0.0980
Epoch 6/20
976/976 [==============================] - 0s 242us/sample - loss: 2.8371 - acc: 0.1240 - val_loss: 2.8114 - val_acc: 0.1306
Epoch 7/20
976/976 [==============================] - 0s 227us/sample - loss: 2.7985 - acc: 0.1393 - val_loss: 2.7845 - val_acc: 0.1184
Epoc


Consider using a TensorFlow optimizer from `tf.train`.


976/976 [==============================] - 0s 435us/sample - loss: 2.7155 - acc: 0.1701 - val_loss: 2.7514 - val_acc: 0.1510
Epoch 11/20
976/976 [==============================] - 0s 236us/sample - loss: 2.7121 - acc: 0.1629 - val_loss: 2.6847 - val_acc: 0.1918
Epoch 12/20
976/976 [==============================] - 0s 177us/sample - loss: 2.6691 - acc: 0.1875 - val_loss: 2.6593 - val_acc: 0.1918
Epoch 13/20
976/976 [==============================] - 0s 182us/sample - loss: 2.6635 - acc: 0.1793 - val_loss: 2.6517 - val_acc: 0.2082
Epoch 14/20
976/976 [==============================] - 0s 193us/sample - loss: 2.6588 - acc: 0.1855 - val_loss: 2.6542 - val_acc: 0.1796
Epoch 15/20
976/976 [==============================] - 0s 232us/sample - loss: 2.6213 - acc: 0.1895 - val_loss: 2.6139 - val_acc: 0.1755
Epoch 16/20
976/976 [==============================] - 0s 190us/sample - loss: 2.5837 - acc: 0.1936 - val_loss: 2.5941 - val_acc: 0.2041
Epoch 17/20
976/976 [==============================] 


Consider using a TensorFlow optimizer from `tf.train`.


976/976 [==============================] - 1s 729us/sample - loss: 2.5411 - acc: 0.2152 - val_loss: 2.5441 - val_acc: 0.1918


#### Saving

In [ ]:
frozen_graph = freeze_session(tf.keras.backend.get_session(), output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, './', "Double input model/double_input_model.pb", as_text=False)